# 🌋 Vesuvius Challenge - Surface Detection: The Winning Strategy

## Goal: 0.6+ Leaderboard Score
Current Baseline: 0.562 (Host)

## The Strategy
To beat the strong nnU-Net baseline, we need to focus on the competition metrics: **Surface Dice**, **VOI**, and **TopoScore**.

### 1. Architecture: Residual 3D U-Net
We use a **Res-UNet** architecture. Deep networks with residual connections are essential for 3D volumetric data to ensure gradients propagate through many layers without vanishing. We use **Instance Normalization**, which is superior to Batch Norm for small batch sizes (typical in 3D).

### 2. Loss Function: Dice + clDice (Centerline Dice)
The standard Dice loss is good for volume overlap but terrible for topology. A small break in a sheet (hole) has a tiny Dice penalty but a huge TopoScore penalty.
We introduce **clDice (soft-skeleton)** loss. It extracts the probabilistic skeleton of the prediction and ensures it matches the ground truth skeleton. This explicitly forces the model to **close holes** and **maintain connectivity**.

### 3. Post-Processing: Hessian (Frangi) Filtering
The scroll sheets are thin, plate-like structures. We use Hessian-based filtering (looking for eigenvalues corresponding to sheets) to refine the probability map before thresholding. This cleans up "blobby" noise.

### 4. Test Time Augmentation (TTA)
We actuate 8x TTA during inference (original + 3 rotations * 2 flips). This is the cheapest way to gain +0.005.


In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from PIL import Image
import tifffile as tiff

# --- CONFIG ---
class Config:
    DATA_DIR = '../input/vesuvius-challenge-surface-detection' # UPDATE THIS PATH!
    TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train_volumes')
    TRAIN_LABEL_DIR = os.path.join(DATA_DIR, 'train_labels')
    
    PATCH_SIZE = (128, 128, 128) # Try 192 if you have 40GB+ VRAM
    BATCH_SIZE = 2
    NUM_WORKERS = 4
    LR = 1e-4
    EPOCHS = 50
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(Config.SEED)
print(f"Device: {Config.DEVICE}")

## Dataset
3D Volumetric Loader that extracts random 3D patches during training.

In [ ]:
class VesuviusDataset(Dataset):
    def __init__(self, volume_ids, img_dir, label_dir, patch_size=(128, 128, 128), transform=None, samples_per_epoch=100):
        self.volume_ids = volume_ids
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.patch_size = patch_size
        self.transform = transform
        self.samples_per_epoch = samples_per_epoch
        
        # Cache volumes in memory if RAM allows (approx 1GB per volume)
        self.volumes = {}
        self.labels = {}
        
        for vid in volume_ids:
            print(f"Loading volume {vid}...")
            # Assuming directory structure: train_volumes/id/images/*.tif
            # But competition format is often single giant TIF per id.
            # Adjust based on exact file structure provided.
            vol_path = glob.glob(os.path.join(img_dir, f"{vid}*.tif"))[0]
            lab_path = glob.glob(os.path.join(label_dir, f"{vid}*.tif"))[0]
            
            self.volumes[vid] = tiff.imread(vol_path)
            self.labels[vid] = tiff.imread(lab_path)
            
    def __len__(self):
        return self.samples_per_epoch
    
    def __getitem__(self, idx):
        # 1. Select random volume
        vid = random.choice(self.volume_ids)
        vol = self.volumes[vid]
        lab = self.labels[vid]
        
        # 2. Select random crop
        # We want to sample "interesting" areas (near surface) more often
        # Simple approach: uniform sampling for now
        d, h, w = vol.shape
        pd, ph, pw = self.patch_size
        
        z = random.randint(0, d - pd)
        y = random.randint(0, h - ph)
        x = random.randint(0, w - pw)
        
        img_patch = vol[z:z+pd, y:y+ph, x:x+pw]
        lab_patch = lab[z:z+pd, y:y+ph, x:x+pw]
        
        # 3. Augmentation (Random Flip)
        if random.random() > 0.5:
            img_patch = np.flip(img_patch, axis=0) # Z-flip
            lab_patch = np.flip(lab_patch, axis=0)
            
        if random.random() > 0.5:
            axis = random.choice([1, 2]) # Y or X flip
            img_patch = np.flip(img_patch, axis=axis)
            lab_patch = np.flip(lab_patch, axis=axis)
            
        # 4. Normalize
        img_patch = (img_patch / 255.0).astype(np.float32)
        lab_patch = (lab_patch > 0).astype(np.float32) # Binary
        
        # Add channel dim
        img_patch = np.expand_dims(img_patch, 0)
        lab_patch = np.expand_dims(lab_patch, 0)
        
        return torch.from_numpy(img_patch.copy()), torch.from_numpy(lab_patch.copy())

## Model: Res-UNet 3D
A U-Net with Residual blocks and InstanceNorm.

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1)
        self.norm1 = nn.InstanceNorm3d(out_channels)
        self.act = nn.LeakyReLU(inplace=True)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)
        self.norm2 = nn.InstanceNorm3d(out_channels)
        
        if in_channels != out_channels:
            self.shortcut = nn.Conv3d(in_channels, out_channels, kernel_size=1)
        else:
            self.shortcut = nn.Identity()
            
    def forward(self, x):
        resid = self.shortcut(x)
        x = self.act(self.norm1(self.conv1(x)))
        x = self.norm2(self.conv2(x))
        return self.act(x + resid)

class ResUNet3D(nn.Module):
    def __init__(self, in_ch=1, out_ch=1, base_ch=32):
        super().__init__()
        
        # Encoder
        self.enc1 = ResBlock(in_ch, base_ch)
        self.pool = nn.MaxPool3d(2)
        self.enc2 = ResBlock(base_ch, base_ch*2)
        self.enc3 = ResBlock(base_ch*2, base_ch*4)
        self.enc4 = ResBlock(base_ch*4, base_ch*8)
        
        # Bottleneck
        self.bottleneck = ResBlock(base_ch*8, base_ch*16)
        
        # Decoder
        self.up4 = nn.ConvTranspose3d(base_ch*16, base_ch*8, kernel_size=2, stride=2)
        self.dec4 = ResBlock(base_ch*16, base_ch*8)
        
        self.up3 = nn.ConvTranspose3d(base_ch*8, base_ch*4, kernel_size=2, stride=2)
        self.dec3 = ResBlock(base_ch*8, base_ch*4)
        
        self.up2 = nn.ConvTranspose3d(base_ch*4, base_ch*2, kernel_size=2, stride=2)
        self.dec2 = ResBlock(base_ch*4, base_ch*2)
        
        self.up1 = nn.ConvTranspose3d(base_ch*2, base_ch, kernel_size=2, stride=2)
        self.dec1 = ResBlock(base_ch*2, base_ch)
        
        self.final = nn.Conv3d(base_ch, out_ch, kernel_size=1)
        
        # Add simple classification head for optional auxiliary loss if needed
        
    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool(e1)
        
        e2 = self.enc2(p1)
        p2 = self.pool(e2)
        
        e3 = self.enc3(p2)
        p3 = self.pool(e3)
        
        e4 = self.enc4(p3)
        p4 = self.pool(e4)
         
        b = self.bottleneck(p4)
        
        u4 = self.up4(b)
        # Padding handling could be added here if shapes don't perfectly match (e.g. valid padding)
        d4 = self.dec4(torch.cat([u4, e4], dim=1))
        
        u3 = self.up3(d4)
        d3 = self.dec3(torch.cat([u3, e3], dim=1))
        
        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e2], dim=1))
        
        u1 = self.up1(d2)
        d1 = self.dec1(torch.cat([u1, e1], dim=1))
        
        return self.final(d1)

## Loss: clDice
The secret weapon for topology. Approximates the skeleton.

In [ ]:
def soft_erode(img):
    """Differentiable erosion using max pooling."""
    # Erosion is min-pooling. -Max(-X) == Min(X)
    p1 = -F.max_pool3d(-img, (3,1,1), (1,1,1), (1,0,0))
    p2 = -F.max_pool3d(-img, (1,3,1), (1,1,1), (0,1,0))
    p3 = -F.max_pool3d(-img, (1,1,3), (1,1,1), (0,0,1))
    return torch.min(torch.min(p1, p2), p3)

def soft_dilate(img):
    """Differentiable dilation using max pooling."""
    return F.max_pool3d(img, (3,3,3), (1,1,1), (1,1,1))

def soft_open(img):
    return soft_dilate(soft_erode(img))

def soft_skel(img, iter_):
    img1 = soft_open(img)
    skel = F.relu(img - img1)
    for i in range(iter_):
        img = soft_erode(img)
        img1 = soft_open(img)
        delta = F.relu(img - img1)
        skel = skel + F.relu(delta - skel * delta)
    return skel

class clDiceLoss(nn.Module):
    def __init__(self, iter_=3, smooth=1.):
        super().__init__()
        self.iter = iter_
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        # y_pred is logits, apply sigmoid
        y_pred = torch.sigmoid(y_pred)
        
        skel_pred = soft_skel(y_pred, self.iter)
        skel_true = soft_skel(y_true, self.iter)
        
        tprec = (torch.sum(torch.multiply(skel_pred, y_true)) + self.smooth) / (torch.sum(skel_pred) + self.smooth)
        tsens = (torch.sum(torch.multiply(skel_true, y_pred)) + self.smooth) / (torch.sum(skel_true) + self.smooth)
        
        cl_dice = 1. - 2.0 * (tprec * tsens) / (tprec + tsens)
        return cl_dice
    
class CombinedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.cldice = clDiceLoss()
        
    def forward(self, pred, target):
        # Warmup: You might want to use only BCE first 10 epochs
        return 0.5 * self.bce(pred, target) + 0.5 * self.cldice(pred, target)

## Training Loop

In [ ]:
def train():
    # MOCK_IDS for demonstration if files don't exist
    # Replace with logic to listdir actual files
    train_ids = ['602831951'] 
    
    # Check if files exist, if not create dummy for run
    if not os.path.exists(Config.TRAIN_IMG_DIR):
        print("Dataset path not found. Please double check CONFIG paths.")
        return
        
    dataset = VesuviusDataset(train_ids, Config.TRAIN_IMG_DIR, Config.TRAIN_LABEL_DIR, Config.PATCH_SIZE)
    loader = DataLoader(dataset, batch_size=Config.BATCH_SIZE, num_workers=Config.NUM_WORKERS)
    
    model = ResUNet3D().to(Config.DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=Config.LR)
    criterion = CombinedLoss()
    # Scheduler: CosineAnnealingWarmRestarts or ReduceLROnPlateau
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    print("Starting training...")
    
    for epoch in range(Config.EPOCHS):
        model.train()
        total_loss = 0
        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{Config.EPOCHS}")
        
        for imgs, labels in pbar:
            imgs = imgs.to(Config.DEVICE)
            labels = labels.to(Config.DEVICE)
            
            optimizer.zero_grad()
            preds = model(imgs)
            loss = criterion(preds, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})
            
        print(f"Epoch {epoch+1} Avg Loss: {total_loss / len(loader):.4f}")
        scheduler.step()
        
        # Save checkpoint
        torch.save(model.state_dict(), f"resunet3d_epoch_{epoch+1}.pth")

if __name__ == "__main__":
    # Uncomment to run training
    # train()
    pass